In [ ]:
from babydragon.chat.memory_chat import FifoVectorChat
from babydragon.oai_utils.utils import *
from babydragon.core.vector_index import MemoryIndex
from babydragon.pandas_integration.pandas_index import PandasIndex
import gradio as gr
import pandas as pd

In [ ]:
import openai
openai.api_key = ""

In [ ]:
class IndexChat(FifoVectorChat):
    def __init__(self, indices, system_prompt=None, name='index_chat', max_context=4000, max_memory=2048, longterm_thread=None, longterm_frac=0.5):
        super().__init__(system_prompt, name, max_memory, longterm_thread, longterm_frac)
        self.indices = indices
        self.max_context = max_context
        self.current_index = None
        self.prompt_func = self.index_memory_prompt
        
    def index_memory_prompt(self, question, k=10):
        prompt = [mark_system(self.system_prompt)]

        if len(self.longterm_thread.memory_thread) > 0 and self.longterm_thread.total_tokens <= self.max_longterm_memory:
            prompt += self.longterm_thread.memory_thread
        elif len(self.longterm_thread.memory_thread) > 0 and self.longterm_thread.total_tokens > self.max_longterm_memory:
            prompt += self.longterm_thread.get_token_bound_prompt(question, k=k)

        prompt += self.memory_thread
        prompt += [mark_question(self.get_index_hints(question, k))]
        return prompt, mark_question(question)

    def get_index_hints(self, question, k=10):
        hints = []
        if self.current_index is not None:
            index_instance = self.indices[self.current_index]
            if isinstance(index_instance, PandasIndex) or isinstance(index_instance, MemoryIndex):
                hints = index_instance.get_token_bound_hints(question, k, self.max_context)
            else:
                raise ValueError("The current index is not a valid index instance.")  
            hints_string = "\n".join(hints)
            hint_prompt = "I am going to ask you a question and you should use the hints to answer it. The hints are:\n{hints_string} . Remember that I can not see the hints, and you should answer without me realizing you are using the hints."
            question_intro = "The question is: {question}"
            return hint_prompt.format(hints_string=hints_string) + question_intro.format(question=question)
        else:
            return question

    def set_current_index(self, index_name):
        if index_name in self.indices:
            self.current_index = index_name
        elif index_name is None:
            self.current_index = None
        else:
            raise ValueError("The provided index name is not available.")
    def reply_from_index(self,query, index_name):
        self.set_current_index(index_name)
        return self.reply(query)


In [ ]:
#create a mathew colville index
from babydragon.pandas_integration.pandas_index import PandasIndex
df_mci = pd.read_csv(r"C:\Users\Tommaso\Documents\Dev\LangDND\DndIndexes\Indexes\matthew_colville_index.csv", converters={"embeddings": eval})
mci = PandasIndex(df_mci, columns=["text"], embeddings_col="embeddings", name="matthew_colville_index")

In [ ]:
#create a srd index
url = r"C:\Users\Tommaso\Documents\Dev\LangDND\DndIndexes\Indexes\srd_index.csv"
df_srd = pd.read_csv(url, converters={"embeddings": eval})
srdi = PandasIndex(df_srd, columns=["values"], embeddings_col="embeddings", name="srd_index")

In [ ]:
## create index dictionary 
indices = {"matthew_colville_index": mci, "srd_index": srdi}

In [ ]:
ic = IndexChat(indices=indices)
ic.max_context=500
ic.max_memory=1400
ic.max_output_tokens= 500

In [ ]:
ic.set_current_index("matthew_colville_index")

In [ ]:
ic.set_current_index("srd_index")

In [ ]:
ic.gradio()